In [ ]:
!pip install torch torchvision --quiet
!pip install transformers  --quiet
!pip install pandas  --quiet
!pip install numpy  --quiet
!pip install sentencepiece  --quiet
!pip install sentence-splitter  --quiet
!pip install shap --quiet





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 9.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Imports**

In [ ]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW,AutoModelForQuestionAnswering, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DebertaTokenizer, DebertaModel, BartTokenizer
import math

# **Model loading**

In [ ]:
# Use a GPU if you have one available (Runtime -> Change runtime type -> GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds for reproducibility
random.seed(26)
np.random.seed(26)
torch.manual_seed(26)

tokenizer = AutoTokenizer.from_pretrained("michiyasunaga/BioLinkBERT-base", do_lower_case=True)

model = AutoModelForSequenceClassification.from_pretrained("michiyasunaga/BioLinkBERT-base")
model.to(device) # Send the model to the GPU if we have one


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at michiyasunaga/BioLinkBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28895, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
import pandas as pd

df=pd.read_csv('./new_data_mutagenicity.csv')

# Apply the function to the column
df['label'] = df['label'].apply(lambda x: x.startswith("['True"))
#df['label'] = df['label'].apply(lambda x: x[0] == "['False - AMES non Mutagenic']")

new_column_names = {'text': 'Abstract', 'label': 'AMES'}
df.rename(columns=new_column_names, inplace=True)


# **Prediction**

In [ ]:
def predict(passage,question):
  sequence = tokenizer.encode_plus(passage,question, return_tensors="pt")['input_ids'].to(device)

  logits = model(sequence)[0]
  probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
  proba_yes = round(probabilities[1], 2)
  proba_no = round(probabilities[0], 2)

  print(f"Question: {question}, Yes: {proba_yes}, No: {proba_no}")

  if (proba_yes >= proba_no):
    return True
  else:
    return False






In [ ]:
filter=list(df.index)

In [ ]:
preds=[]
index=[]
for i in filter:
    try:
      passage=df.Abstract[i]
      question =df.questions[i]
      answer=predict(passage,question)
      preds.append(answer)
      index.append(i)
    except Exception as e:
      print(e)
      continue



Question: Is AZS mutagenic for TA98 or TA100 strain?, Yes: 0.6, No: 0.4
Question: Is Melamine mutagenic for TA98 or TA100 or TA1535 or TA102 strain?, Yes: 0.62, No: 0.38
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.64, No: 0.36
Question: Is 1-NP mutagenic for TA98 or TA100 or TA1535 strain?, Yes: 0.67, No: 0.33
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.61, No: 0.39
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.64, No: 0.36
Question: Is 1-NP mutagenic for TA98 or TA100 strain?, Yes: 0.65, No: 0.35
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.59, No: 0.41
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.61, No: 0.39
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.63, No: 0.37
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.66, No: 0.34
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.63, No: 0.37
Question: Is 1-NP mutagenic for TA98 or TA100 strain?, Yes: 0.64, No: 0.36
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.63, No: 0.37
Q

In [ ]:
len(index)


1481

In [ ]:
df=df[df.index.isin(index)]
true_results=df['AMES'].tolist()

In [ ]:
#With Deberta model w no maintenance of equal samples of the two classes for 3 epochs
from sklearn.metrics import classification_report
print(classification_report(true_results, preds,output_dict=True))

{'False': {'precision': 1.0, 'recall': 0.0015105740181268882, 'f1-score': 0.0030165912518853692, 'support': 662}, 'True': {'precision': 0.5533783783783783, 'recall': 1.0, 'f1-score': 0.7124836885602436, 'support': 819}, 'accuracy': 0.5536799459824443, 'macro avg': {'precision': 0.7766891891891892, 'recall': 0.5007552870090635, 'f1-score': 0.35775013990606447, 'support': 1481}, 'weighted avg': {'precision': 0.7530161322700148, 'recall': 0.5536799459824443, 'f1-score': 0.39535524938527183, 'support': 1481}}


In [ ]:
#With Deberta model w no maintenance of equal samples of the two classes for 3 epochs
from sklearn.metrics import classification_report
print(classification_report(true_results, preds))

              precision    recall  f1-score   support

       False       1.00      0.00      0.00       662
        True       0.55      1.00      0.71       819

    accuracy                           0.55      1481
   macro avg       0.78      0.50      0.36      1481
weighted avg       0.75      0.55      0.40      1481

